In [ ]:
import pandas as pd
df = pd.read_csv('augDATA.csv')

In [ ]:
df.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def stringToImage(string):
    img = plt.imread(string)
    return img

def plotFaces(nrows=5, ncols=5):
    selection = np.random.choice(df.index, size=(nrows*ncols), replace=False)
    image_strings = df.loc[selection]['image_path']
    fig, axes = plt.subplots(figsize=(10, 10), nrows=nrows, ncols=ncols)
    for string, ax in zip(image_strings, axes.ravel()):
        ax.imshow(stringToImage(string), cmap='gray')
        ax.axis('off')

In [ ]:
keypoint_cols = list(df.columns)[:-1]
xy = df.iloc[0][keypoint_cols].values.reshape((17, 3))

In [ ]:

plt.imshow(stringToImage(df.iloc[0]['image_path']), cmap='gray')

In [ ]:
plt.plot(xy[:, 0], xy[:, 1], 'ro')
plt.imshow(stringToImage(df.iloc[0]['image_path']), cmap='gray')

In [ ]:
df.describe().loc['count'].plot.bar()

In [ ]:
df.shape

In [ ]:
X = np.stack([stringToImage(string) for string in df['image_path']]).astype(np.float)[:, :, :]

In [ ]:
X.shape, X.dtype

In [ ]:
Y = np.vstack(df[df.columns[:-1]].values.astype(np.float))

In [ ]:
Y.shape, Y.dtype

In [ ]:
x_train = X/255.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

output_pipe = make_pipeline(
    MinMaxScaler(feature_range=(-1, 1))
)

y_train = output_pipe.fit_transform(Y)

In [ ]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, MaxPooling2D,Dropout, Flatten, Dense, GlobalAveragePooling2D

In [ ]:
# model = Sequential()
# model.add(BatchNormalization(input_shape=(112, 112, 3)))
# model.add(Conv2D(32, (9, 9), input_shape=(104, 104, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0))

# model.add(Conv2D(32, (5, 5), input_shape=(48,48,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0))

# model.add(Conv2D(16, (5, 5), input_shape=(20, 20, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Flatten())

# model.add(Dense(51))
#model.summary()

In [ ]:
model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape=(112, 112, 3)) )
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(32, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(64, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Flatten()) 
model.add(Dense(51)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(51)) 
model.add(Activation('sigmoid')) 

In [ ]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mse', metrics=['accuracy'])
epochs = 1000
history = model.fit(x_train, y_train, 
                 validation_split=0.2, shuffle=True, 
                 epochs=epochs)

In [ ]:
model.save('model.h5')

In [ ]:
plt.plot(history.history['accuracy'])

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
import os
os.sys.path